<a href="https://colab.research.google.com/github/prachigupta2006/feature-engineering/blob/main/feature_construction_feature_splitting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import scipy.stats as stats

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

from sklearn.preprocessing import KBinsDiscretizer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer

In [2]:
from google.colab import files
uploaded = files.upload()

Saving Titanic-Dataset.csv to Titanic-Dataset.csv


In [5]:
df= pd.read_csv('Titanic-Dataset.csv')[['Age','Pclass','SibSp','Parch','Survived']]

In [7]:
df.head()

,Age,Pclass,SibSp,Parch,Survived
0,22.0,3,1,0,0
1,38.0,1,1,0,1
2,26.0,3,0,0,1
3,35.0,1,1,0,1
4,35.0,3,0,0,0


In [10]:
df.isnull().sum()

,0
Age,177
Pclass,0
SibSp,0
Parch,0
Survived,0


In [11]:
df.dropna(inplace=True)

In [12]:
df.head()

,Age,Pclass,SibSp,Parch,Survived
0,22.0,3,1,0,0
1,38.0,1,1,0,1
2,26.0,3,0,0,1
3,35.0,1,1,0,1
4,35.0,3,0,0,0


### TRAIN TEST SPLIT

In [16]:
x=df.drop(columns=['Survived'])
y=df['Survived']

In [18]:
np.mean(cross_val_score(LogisticRegression(),x,y,scoring='accuracy',cv=20))

0.6933333333333332

# **Apply Feature Construction**

In [21]:
x['Family'] = x['SibSp'] + x['Parch'] +1

In [22]:
x.head()

,Age,Pclass,SibSp,Parch,Family
0,22.0,3,1,0,2
1,38.0,1,1,0,2
2,26.0,3,0,0,1
3,35.0,1,1,0,2
4,35.0,3,0,0,1


apply a function on this new column

In [25]:
def myfunc(num):
  if num ==1:
    #alone
    return 0
  elif num >1 and num <=4:
    #small family
    return 1
  else:
    #large family
    return 2

In [26]:
myfunc(4)

1

In [27]:
x['family type'] =x['Family'].apply(myfunc)

In [28]:
x.head()

,Age,Pclass,SibSp,Parch,Family,family type
0,22.0,3,1,0,2,1
1,38.0,1,1,0,2,1
2,26.0,3,0,0,1,0
3,35.0,1,1,0,2,1
4,35.0,3,0,0,1,0


In [29]:
x.drop(columns=['SibSp','Parch','Family'],inplace=True)

In [30]:
np.mean(cross_val_score(LogisticRegression(),x,y,scoring='accuracy',cv=20))

0.7003174603174602

# **Feature Splitting**

In [37]:
from google.colab import files
uploaded = files.upload()

Saving Titanic-Dataset.csv to Titanic-Dataset (1).csv


In [38]:
df= pd.read_csv('Titanic-Dataset.csv')

In [39]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [40]:
df['Name']

,Name
0,"Braund, Mr. Owen Harris"
1,"Cumings, Mrs. John Bradley (Florence Briggs Th..."
2,"Heikkinen, Miss. Laina"
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)"
4,"Allen, Mr. William Henry"
...,...
886,"Montvila, Rev. Juozas"
887,"Graham, Miss. Margaret Edith"
888,"Johnston, Miss. Catherine Helen ""Carrie"""
889,"Behr, Mr. Karl Howell"


Through this we first split the whole name by a comma "," making it from "Braund, Mr. Owen Harris" to "Mr. Owen Harris" and the further splitted this by a fullstop "." and from "Mr. Owen Harris" to it seperated to "Mr." and "Owen Harris" and indexing the splitted through an index of assigning "Mr." to [0] and "Owen Harris" to [1] by a diffrent columns named "title" and "Name".

In [41]:
df['title'] = df['Name'].str.split(',',expand=True)[1].str.split('.',expand=True)[0]

In [42]:
df[['title','Name']]

,title,Name
0,Mr,"Braund, Mr. Owen Harris"
1,Mrs,"Cumings, Mrs. John Bradley (Florence Briggs Th..."
2,Miss,"Heikkinen, Miss. Laina"
3,Mrs,"Futrelle, Mrs. Jacques Heath (Lily May Peel)"
4,Mr,"Allen, Mr. William Henry"
...,...,...
886,Rev,"Montvila, Rev. Juozas"
887,Miss,"Graham, Miss. Margaret Edith"
888,Miss,"Johnston, Miss. Catherine Helen ""Carrie"""
889,Mr,"Behr, Mr. Karl Howell"


In [43]:
df['title']

,title
0,Mr
1,Mrs
2,Miss
3,Mrs
4,Mr
...,...
886,Rev
887,Miss
888,Miss
889,Mr


In [64]:
df['is_married'] = 0
df['is_married'].loc[df['title'] == 'Mrs'] = 1

<ipython-input-64-fa5f573277db>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['is_married'].loc[df['title'] == 'Mrs'] = 1
